# Automated Topic Summary Page Generation

## 1. Project Introduction

In [2]:
# import libraries


## 2. Crawl the news

input: key word, start time,end time
<br/>output:raw_news.json
<br/>beautiful soup

In [ ]:
# raw data structure
raw_data_list = []
raw_data = {
    "title" : title
    "date" : date
    "link" : link
    "text" : text
}

## 3. Clean the data

input: raw_news.json
<br/>output: clean_news.json

In [ ]:
# cleaned data structure
cleaned_data_list = []
cleaned_data = {
    "title" : title
    "date" : date
    "link" : link
    "text" : text
}

## 4. Extract the information

input:clean_news.json
<br/>output:timeline.json, entities.json
<br/>NLP

In [ ]:
# timeline structure
timeline_list = []
timeline = {
    "title" : title
    "date" : date
}

# entities.json
entities_list = []

entity = {
    "names" : names
    "locations" : locations
    "organizations" organizaitions
}

## 5. Summarize

input:clean_news.json
<br/>output:summary(string/text)
<br/>LLM API

## 6. Generate HTML Page
include **a main summary**, **a list of key entities**, **a timeline of major developments**, and **links to the original source articles**

input: summary,entities.json,timeline.json,cleaned_data.json(link)
<br/>output:summary_page.html

## 7. Run pipeline

In [ ]:
# Input event name:
event_name = "" # The 2024 Nobel Prize
# Input start time:
start_time = "" # “2023-9-1” 
# Input end time:
end_time = "" # “2024-12-10”


In [ ]:
# Crawl the news

# Clean the data

# Extract information

# Summarize

# Generate HTML page
summary_page = ""